In [64]:
import os
import json
import pandas as pd



BIDS_DIR = '/Volumes/vosslabhpc/Projects/BikeExtend/3-Experiment/2-Data/BIDS'
SOURCE_DIR = '/Volumes/vosslabhpc/Projects/BikeExtend/3-Experiment/2-Data/BIDS/sourcedata'

In [65]:
csv = pd.read_csv('/Volumes/vosslabhpc/Projects/BikeExtend/3-Experiment/2-Data/BIDS/derivatives/datacheck/EXTEND_jsonCHECK_S24.csv')



In [66]:
# if any of columns B through R are less than 86 but not 0, add subject (column A) to list

bad_subs = ['sub-999']
'''
for i in range(len(csv)):
    if csv.iloc[i, 1] < 86 and csv.iloc[i, 1] != 0:
        bad_subs.append(csv.iloc[i, 0]
                          )
'''

'\nfor i in range(len(csv)):\n    if csv.iloc[i, 1] < 86 and csv.iloc[i, 1] != 0:\n        bad_subs.append(csv.iloc[i, 0]\n                          )\n'

In [67]:
bad_subs

['sub-999']

In [68]:
BIDS_DIR = '/Volumes/vosslabhpc/Projects/BikeExtend/3-Experiment/2-Data/BIDS'
SOURCE_DIR = '/Volumes/vosslabhpc/Projects/BikeExtend/3-Experiment/2-Data/BIDS/sourcedata'

In [69]:
# for all subjects in bad_subs, enter the following directories and store the path to all json files
# in each directory, enter either func or dwi folders to look for jsons
dirs = ['ses-day1pre', 'ses-day1post', 'ses-day2pre', 'ses-day2post', 'ses-post']

jsons = []

for sub in bad_subs:
    #first enter the sub directory
    sub_dir = os.path.join(BIDS_DIR, sub)
    #then enter each directory in dirs
    for d in dirs:
        d_dir = os.path.join(sub_dir, d)
        #then enter either func or dwi, if they exist
        if os.path.exists(os.path.join(d_dir, 'func')):
            func_dir = os.path.join(d_dir, 'func')
            for file in os.listdir(func_dir):
                if file.endswith('.json'):
                    jsons.append(os.path.join(func_dir, file))
        if os.path.exists(os.path.join(d_dir, 'dwi')):
            dwi_dir = os.path.join(d_dir, 'dwi')
            for file in os.listdir(dwi_dir):
                if file.endswith('.json'):
                    jsons.append(os.path.join(dwi_dir, file))

In [70]:
jsons

['/Volumes/vosslabhpc/Projects/BikeExtend/3-Experiment/2-Data/BIDS/sub-999/ses-day1pre/func/sub-999_ses-day1pre_task-breathhold_bold.json',
 '/Volumes/vosslabhpc/Projects/BikeExtend/3-Experiment/2-Data/BIDS/sub-999/ses-day1pre/func/sub-999_ses-day1pre_task-rest_run-1_bold.json',
 '/Volumes/vosslabhpc/Projects/BikeExtend/3-Experiment/2-Data/BIDS/sub-999/ses-day1pre/dwi/sub-999_ses-day1pre_dwi.json']

In [71]:
# In SOURCE_DIR, go through all directories that start with subs in bad_subs
# in each directory, if has ses-day1pre, scan the following directory /scans and save path for folders that start with 4,5,6,7
#if directory has ses-day2pre, scan the following directory /scans and save path for folders that start with 4,5,6,7
# if directory has ses-day1post or day2post, scan the following directory /scans and save path for folders that start with 4,5,6
# if directory has ses-post, scan the following directory /scans and save path for folders that start with 4,5,6,7,10
#s.startswith('4-') or s.startswith('5-') or s.startswith('6-') or

folders = []

for sub in bad_subs:
    for d in os.listdir(SOURCE_DIR):
        if d.startswith(sub):
            if d == sub + '_ses-day1pre':
                d_dir = os.path.join(SOURCE_DIR, d)
                for s in os.listdir(os.path.join(d_dir, 'scans')):
                    #replace here
                    if  s.startswith('7-'):
                        folders.append(os.path.join(d_dir, 'scans', s))
            if d == sub + '_ses-day2pre':
                d_dir = os.path.join(SOURCE_DIR, d)
                for s in os.listdir(os.path.join(d_dir, 'scans')):
                    if s.startswith('4-') or s.startswith('5-') or s.startswith('6-') or s.startswith('7-'):
                        folders.append(os.path.join(d_dir, 'scans', s))
            if d == sub + '_ses-day1post' or d == sub + '_ses-day2post':
                d_dir = os.path.join(SOURCE_DIR, d)
                for s in os.listdir(os.path.join(d_dir, 'scans')):
                    if s.startswith('4-') or s.startswith('5-') or s.startswith('6-'):
                        folders.append(os.path.join(d_dir, 'scans', s))
            if d == sub + '_ses-post':
                d_dir = os.path.join(SOURCE_DIR, d)
                for s in os.listdir(os.path.join(d_dir, 'scans')):
                    if s.startswith('4-') or s.startswith('5-') or s.startswith('6-') or s.startswith('7-') or s.startswith('10-'):
                        folders.append(os.path.join(d_dir, 'scans', s))


folders

['/Volumes/vosslabhpc/Projects/BikeExtend/3-Experiment/2-Data/BIDS/sourcedata/sub-999_ses-day1pre/scans/7-dwi']

In [72]:
# for each in folders add /resources/DICOM/files to the end of the path
dicoms = []
for f in folders:
    dicoms.append(os.path.join(f, 'resources/DICOM/files'))

dicoms

['/Volumes/vosslabhpc/Projects/BikeExtend/3-Experiment/2-Data/BIDS/sourcedata/sub-999_ses-day1pre/scans/7-dwi/resources/DICOM/files']

In [73]:
# for each in dicoms, find the output path in the BIDS directory for that subject, session and scan

output_path = []

for d in dicoms:    

    # get the subject and session
    sub = d.split('/')[-6].split('_')[0]
    ses = d.split('/')[-6].split('_')[1]
    scan = d.split('/')[-4]


    #if scan contains func, then output path is /func
    #if scan contains breathold, output path is /func
    #if scan contains dwi, then output path is /dwi

    if 'func' in scan or 'breathhold' in scan:
        output_path.append(os.path.join(BIDS_DIR, sub, ses, 'func'))
    if 'dwi' in scan:
        output_path.append(os.path.join(BIDS_DIR, sub, ses, 'dwi'))
    
output_path


['/Volumes/vosslabhpc/Projects/BikeExtend/3-Experiment/2-Data/BIDS/sub-999/ses-day1pre/dwi']

In [78]:
csv = pd.read_csv('/Volumes/vosslabhpc/Projects/BikeExtend/3-Experiment/2-Data/BIDS/derivatives/datacheck/EXTEND_jsonCHECK_S24.csv')



# if any of columns B through R are less than 86 but not 0, add subject (column A) to list

bad_subs = []
for i in range(len(csv)):
    if csv.iloc[i, 1] < 86 and csv.iloc[i, 1] != 0:
        bad_subs.append(csv.iloc[i, 0]
                          )
#save bad_subs to a file

with open('/Volumes/vosslabhpc/Projects/BikeExtend/3-Experiment/2-Data/BIDS/derivatives/Zak/code/json_qc_s-24/bad_subs.txt', 'w') as f:
    for sub in bad_subs:
        f.write(sub + '\n')